before start the testing, see Readme, be sure the IPbus, control Hub and python 3.8 is installed.
To properly set all paths run source setup.sh.

## Cell 1: KCU Connection Setup
This cell establishes connection to the KCU board and verifies basic communication.
- Sets KCU IP address to 192.168.0.10
- Enables control hub mode
- Performs firmware version check and loopback test

**Expected Output:** 
- "Successfully connected to KCU"
- Firmware version information
- "KCU Loopback test PASSED"


In [2]:
from tamalero.utils import get_kcu

kcu_ip = "192.168.0.10" ## If your KCU ip is diff, modify it.

kcu = get_kcu(
    kcu_ip,
    control_hub=True,
    host='localhost',
    verbose=False
)
print("Successfully connected to KCU.")

# Check the KCU's status and firmware
#kcu.status() # Prints LpGBT link statuses from KCU 
fw_ver = kcu.get_firmware_version(verbose=True) #
kcu.check_clock_frequencies(verbose=True) # Verifies KCU clock stability

# Perform a simple loopback register test to confirm communication
loopback_val = 0xABCD1234
kcu.write_node("LOOPBACK.LOOPBACK", loopback_val) #
read_val = kcu.read_node("LOOPBACK.LOOPBACK").value()
if read_val == loopback_val:
    print(f"KCU Loopback test PASSED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")
else:
    print(f"KCU Loopback test FAILED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")

IPBus address: chtcp-2.0://localhost:10203?target=192.168.0.10:50001
KCU firmware version: 0.0.0
Successfully connected to KCU.
Firmware version: 2025/05/29 29:14:14 v0.0.0 sha=cc2cf66
0x100B  r       FW_INFO.CLK125_FREQ                               0x07735940
0x1008  r       FW_INFO.CLK320_FREQ                               0x131C884F
0x1007  r       FW_INFO.CLK_40_FREQ                               0x02639109
0x1009  r       FW_INFO.REFCLK_FREQ                               0x131C884F
0x1018  r       FW_INFO.RXCLK0_FREQ                               0x131C884F
0x1019  r       FW_INFO.RXCLK1_FREQ                               0x12861947
0x100E  r       FW_INFO.TXCLK0_FREQ                               0x131C884F
0x100F  r       FW_INFO.TXCLK1_FREQ                               0x131C884F
KCU Loopback test PASSED: Wrote 0xABCD1234, Read 0xABCD1234


This cell configures the ReadoutBoard object and establishes lpGBT communication.

Steps performed:
1. Initialize ReadoutBoard with modulev1 configuration
2. Detect and verify readout board version
3. Check lpGBT link status
4. Verify no FEC errors on DAQ link

Troubleshooting:
- If version detection fails, check lpGBT connection
- If FEC errors > 0, reset error counters and check links


In [3]:
from tamalero.ReadoutBoard import ReadoutBoard

rb = ReadoutBoard(
    rb=0,
    kcu=kcu,
    config="modulev1", 
    trigger=False,     
    verbose=True
)

print(f"Readout Board version detected: {rb.ver}")

0x1d7 readback value is 0xae
lpGBT version found True
 > lpGBT v2 detected
LPGBT.RWF.EPORTRX.EPRXECENABLE: 1
LPGBT.RWF.EPORTRX.EPRXECAUTOPHASERESETDISABLE: 0
LPGBT.RWF.EPORTRX.EPRXECTRACKMODE: 0
LPGBT.RWF.EPORTRX.EPRXECPHASESELECT: 0
LPGBT.RWF.EPORTRX.EPRXECINVERT: 0
LPGBT.RWF.EPORTRX.EPRXECACBIAS: 0
LPGBT.RWF.EPORTRX.EPRXECTERM: 1
LPGBT.RWF.EPORTRX.EPRXECPULLUPENABLE: 0
LPGBT.RWF.EPORTTX.EPTXECDRIVESTRENGTH: 3
LPGBT.RWF.EPORTTX.EPTXECINVERT: 0
LPGBT.RWF.EPORTTX.EPTXECTRISTATE: 0
LPGBT.RWF.EPORTTX.EPTXECENABLE: 1
LPGBT.RWF.EPORTCLK.EPCLK28LOWRES: 0
LPGBT.RWF.EPORTCLK.EPCLK28INVERT: 1
LPGBT.RWF.EPORTCLK.EPCLK28DRIVESTRENGTH: 4
LPGBT.RWF.EPORTCLK.EPCLK28FREQ: 1
LPGBT.RWF.EPORTCLK.EPCLK28PREEMPHASISSTRENGTH: 0
LPGBT.RWF.EPORTCLK.EPCLK28PREEMPHASISMODE: 0
LPGBT.RWF.EPORTCLK.EPCLK28PREEMPHASISWIDTH: 0
 > Readout Board version detected: 2
No FEC errors detected on DAQ link
Readout Board version detected: 2


- If FEC errors > 0, no worries, READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT was reset later
READOUT_BOARD_0.LPGBT.UPLINK_1.FEC_ERR_CNT is red due to our current setup only have one uplink which is uplink 0

In [3]:
print("Checking DAQ LpGBT base configuration:")
rb.DAQ_LPGBT.read_base_config() #

# Read on-board temperatures
print("\nReading temperatures:")
rb.read_temp(verbose=True) #

print("\nReading DAQ LpGBT ADCs:")
try:
    rb.DAQ_LPGBT.read_adcs(check=True, strict_limits=True) #
except ValueError as e:
    print(f"LpGBT ADC check issue: {e}")


print(f"\nDAQ LpGBT Link Good: {rb.DAQ_LPGBT.link_status()}") 
fec_errors = rb.get_FEC_error_count() # Reads KCU registers defined in READOUT_BOARD.xml
print(f"FEC Errors: DAQ LpGBT = {fec_errors.get('DAQ', 'N/A')}")



Checking DAQ LpGBT base configuration:
Register                                                                        value     default   
LPGBT.RWF.CALIBRATION.VREFENABLE                                                1         1         
LPGBT.RWF.CALIBRATION.VREFTUNE                                                  99        99        
LPGBT.RWF.CLOCKGENERATOR.CLKGCALIBRATIONENDOFCOUNT                              14        14        
LPGBT.RWF.CLOCKGENERATOR.CLKGBIASGENCONFIG                                      8         8         
LPGBT.RWF.CLOCKGENERATOR.CDRCONTROLOVERRIDEENABLE                               0         0         
LPGBT.RWF.CLOCKGENERATOR.CLKGDISABLEFRAMEALIGNERLOCKCONTROL                     0         0         
LPGBT.RWF.CLOCKGENERATOR.CLKGCDRRES                                             1         1         
LPGBT.RWF.CLOCKGENERATOR.CLKGVCORAILMODE                                        1         1         
LPGBT.RWF.CLOCKGENERATOR.CLKGVCODAC                 

Init ETROC2, our etroc2 i2c address is 0x63 and i2c channel is 1, modify it if yours diff

In [4]:
from tamalero.ETROC import ETROC
from tamalero.colors import green, red, yellow
from tamalero.LPGBT import LPGBT
import time

etroc_elinks_map = {0: [0, 4, 8, 12]} # should map your design
print("Initializing ETROC object (I2C Addr: 0x63, Channel: 1)...")
try:
    my_etroc2 = ETROC(
        rb,
        master='lpgbt',
        i2c_adr=0x63,                 
        i2c_channel=1,
        elinks=etroc_elinks_map,
        strict=False,
        verbose=True 
    )

    if not my_etroc2.is_connected():
        raise ConnectionError("Failed to connect to ETROC2 via I2C.")

    print(green("SUCCESS: I2C communication with ETROC2 is established."))
    
    print("-" * 50)
    print("Verifying a key configuration register...")
    register_to_check = 'disScrambler'
    expected_value = 1
    
    read_back_value = my_etroc2.rd_reg(register_to_check)
    print(f"Reading back '{register_to_check}' register...")
    print(f"  - Value Expected: {expected_value}")
    print(f"  - Value Read Back: {read_back_value}")

    if read_back_value == expected_value:   ## check etroc register can be read correctly
        print(green("  - SUCCESS: Register read-back matches expected value."))
    else:
        print(red("  - FAILURE: Register read-back does NOT match expected value."))
    
    # print("-" * 50)
    # print("Next Step: Diagnosing the E-Link Lock Failure")
    # print("-" * 50)
    # register_to_test = 'BCIDoffset'
    # test_value = 0xAB 
    
    # print(f"  - Reading initial value of '{register_to_test}'...")
    # initial_value = my_etroc2.rd_reg(register_to_test)
    # print(f"  - Initial value: {hex(initial_value)}")
    
    # print(f"  - Writing test value {hex(test_value)} to '{register_to_test}'...")
    # my_etroc2.wr_reg(register_to_test, test_value)
    
    # print(f"  - Reading back value from '{register_to_test}'...")
    # read_back_value = my_etroc2.rd_reg(register_to_test)
    # print(f"  - Value read back: {hex(read_back_value)}")
    
    # if read_back_value == test_value:
    #     print(green("  - SUCCESS: Manual write/read test passed! I2C writes are working."))
    # else:
    #     print(red("  - FAILURE: Manual write/read test failed. I2C writes may not be working correctly."))
except Exception as e:
    print(red(f"An error occurred during register read-back: {e}"))

Initializing ETROC object (I2C Addr: 0x63, Channel: 1)...
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
elinks not locked, resetting PLL and FC modules
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
SUCCESS: I2C communication with ETROC2 is established.
--------------------------------------------------
Verifying a key configuration register...
4
Reading back 'disScrambler' register...
  - Value Expected: 1
  - Value Read Back: 1
  - SUCCESS: Register read-back matches expected value.


Just checking etroc status and see which elink is locked.
if elink is unlock, can't retrieve data from etroc

In [5]:
print(f"ETROC connected: {my_etroc2.is_connected()}")
print(f"ETROC version: {my_etroc2.get_ver()}")

# Check if ETROC is in a good state
if hasattr(my_etroc2, 'is_good'):
    print(f"ETROC is good: {my_etroc2.is_good()}")

# Check controller state
if hasattr(my_etroc2, 'controllerState'):
    state = my_etroc2.rd_reg("controllerState")
    print(f"Controller state: {state} (should be 11)")

# Check Pll lock status
Pll_state = my_etroc2.rd_reg("pllUnlockCount")
print(f"PLL Unlock Count: {Pll_state} (should be 0 or very few and would not increase)")

# Check elink lock status
print("\n=== Elink Status ===")
elink_status = my_etroc2.get_elink_status()
print(f"Elink status: {elink_status}")


for elink in [0, 4, 8, 12]: 
    locked = rb.etroc_locked(elink, slave=False)
    print(f"Elink {elink} locked: {locked}")
    
    if locked:
        packet_count = rb.read_packet_count(elink, slave=False)
        error_count = rb.read_error_count(elink, slave=False)
        filler_rate = rb.read_filler_rate(elink, slave=False)
        
        print(f"  Packets: {packet_count}")
        print(f"  Errors: {error_count}")
        print(f"  Filler rate: {filler_rate}")
        
print('-' * 50)
eprx_group_to_check = [0,1,2,3]
    
try:
    for group in eprx_group_to_check:
        # Read the Read-Only status register
        dll_locked_status = rb.DAQ_LPGBT.rd_reg(f"LPGBT.RO.EPORTRX_RO.EPRX.EPRX{group}DLLLOCKED")
        if dll_locked_status == 1:
            print(green(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (Locked)"))
        else:
            print(red(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (NOT Locked)"))
except Exception as e:
    print(red(f"dddCould not read LpGBT EPRX DLL status. Error: {e}"))


4
ETROC connected: 44
4
4
4
ETROC version: 0-0-0
4
ETROC is good: True
4
Controller state: 11 (should be 11)
4
4
PLL Unlock Count: 5 (should be 0 or very few and would not increase)

=== Elink Status ===
Elink status: {0: [False, False, False, True]}
Elink 0 locked: False
Elink 4 locked: False
Elink 8 locked: False
Elink 12 locked: True
  Packets: 65535
  Errors: 65535
  Filler rate: 16777215
--------------------------------------------------
  - LpGBT EPRX Group 0 DLL Lock Status: 1 (Locked)
  - LpGBT EPRX Group 1 DLL Lock Status: 1 (Locked)
  - LpGBT EPRX Group 2 DLL Lock Status: 0 (NOT Locked)
  - LpGBT EPRX Group 3 DLL Lock Status: 0 (NOT Locked)


check uplink status and rese FEC errors if not 0

In [5]:
print("=== Check Uplink Status ===")

# Check uplink error counts
uplink0_errors = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.LPGBT.UPLINK_0.FEC_ERR_CNT").value()

print(f"Uplink 0 FEC errors: {uplink0_errors}")

# Check uplink ready status
uplink0_ready = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.LPGBT.UPLINK_0.READY").value()

print(f"Uplink 0 ready: {uplink0_ready}")

# Reset FEC error counters
print("\nResetting FEC error counters...")
rb.reset_FEC_error_count()

print("\nChecking elink 12 status again...")
rb.reset_data_error_count()
time.sleep(1)

for elinks in [12]:  # we current only have elink 12 locked
    locked = rb.etroc_locked(12, slave=False)
    packet_count = rb.read_packet_count(elinks, slave=False)
    error_count = rb.read_error_count(elinks, slave=False)
    filler_rate = rb.read_filler_rate(elinks, slave=False)

    print(f"Elink {elinks} - Locked: {locked}, Packets: {packet_count}, Errors: {error_count}, Filler: {filler_rate}")

    if filler_rate > 15000000 and error_count < 100:
        print(f"Elink {elinks} appears to be working!")
    else:
        print(f"Elink {elinks} still has issues")

=== Check Uplink Status ===
Uplink 0 FEC errors: 0
Uplink 0 ready: 1

Resetting FEC error counters...
Error counts before reset:
Address Perm.   Name                                              Value   
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT        0x00000000
0x2011  r       READOUT_BOARD_0.LPGBT.UPLINK_1.FEC_ERR_CNT        0x0000FFFF
Error counts after reset:
Address Perm.   Name                                              Value   
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT        0x00000000
0x2011  r       READOUT_BOARD_0.LPGBT.UPLINK_1.FEC_ERR_CNT        0x0000FFFF

Checking elink 12 status again...
Elink 12 - Locked: True, Packets: 0, Errors: 0, Filler: 16777215
Elink 12 appears to be working!


just make sure lpGBT assignment is correct

In [7]:
print("=== Verify lpGBT Assignment ===")

print("Checking master vs slave elink locks...")

# Check master elinks (lpgbt=0)
master_locked = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.ETROC_LOCKED").value()
print(f"Master (lpgbt=0) elinks locked: {bin(master_locked)}")

# Check slave elinks (lpgbt=1) 
slave_locked = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.ETROC_LOCKED_SLAVE").value()
print(f"Slave (lpgbt=1) elinks locked: {bin(slave_locked)}")

for elink in [12, 14]:
    print(f"\nElink {elink}:")

    locked_master = rb.etroc_locked(elink, slave=False)
    print(f"  lpgbt=0: {locked_master}")
    
    locked_slave = rb.etroc_locked(elink, slave=True)
    print(f"  lpgbt=1: {locked_slave}")
    
    if locked_master:
        print(f"  → Elink {elink} is on lpgbt=0 (DAQ)")
    elif locked_slave:
        print(f"  → Elink {elink} is on lpgbt=1 (Trigger)")
    else:
        print(f"  → Elink {elink} not locked on either lpGBT")

print(f"\n2. Your ETROC elinks configuration:")
print(f"elinks[0]: {my_etroc2.elinks.get(0, 'Not configured')}")
print(f"elinks[1]: {my_etroc2.elinks.get(1, 'Not configured')}")

=== Verify lpGBT Assignment ===
Checking master vs slave elink locks...
Master (lpgbt=0) elinks locked: 0b1000000000000
Slave (lpgbt=1) elinks locked: 0b0

Elink 12:
  lpgbt=0: True
  lpgbt=1: False
  → Elink 12 is on lpgbt=0 (DAQ)

Elink 14:
  lpgbt=0: False
  lpgbt=1: False
  → Elink 14 not locked on either lpGBT

2. Your ETROC elinks configuration:
elinks[0]: [0, 4, 8, 12]
elinks[1]: Not configured


try to reproduce data from ETROC without charge injection
under workmode : 0(normal mode) data type should be only contain header and trailer only

In [7]:
from tamalero.FIFO import FIFO
from tamalero.DataFrame import DataFrame

print("\n=== Get data without charge injection ===")
df = DataFrame()
fifo = FIFO(rb)

fifo.reset()
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()  
fifo.use_etroc_data()
time.sleep(1)

rb.DAQ_LPGBT.set_uplink_group_data_source("normal") 
my_etroc2.wr_reg("QInjEn", 0, broadcast=True)   ## ensure no charge injection
my_etroc2.wr_reg("workMode", 0, broadcast=True) ## workMode == 0 : normal mode else self-test
my_etroc2.wr_reg("disDataReadout", 0, broadcast=True) 
my_etroc2.wr_reg("singlePort", 1)       
my_etroc2.disable_QInj(broadcast= True) ## ensure no charge injection

fifo.send_l1a(4096)  ## l1a value based on test_ETROC.py

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    if occupancy > 0:
        print(green("SUCCESS: Data is being generated!"))
        print(f"   FIFO returned {occupancy} data items")

        for j, word in enumerate(data[:10]):
            data_type, event_data = word[0], word[1]
            print(f"  Event {j}: {data_type} -> {event_data}")
           
except Exception as e:
    print(red(f"Read failed: {e}"))


# fifo.set_trigger_rate(1000)
# time.sleep(5)
# fifo.set_trigger_rate(0)


# occupancy = fifo.get_occupancy()
# lost_words = fifo.get_lost_word_count()

# print(f"Occupancy after continuous trigger: {occupancy}")
# print(f"Lost words: {lost_words}")
# time.sleep(1)


# if occupancy > 0 or lost_words > 0:
#     print("SUCCESS: Data is being generated!")
#     try:
#         # Use the simple read method
#         raw_data = fifo.read(dispatch=True)
#         print(f"✅ Successfully read {len(raw_data)} words!")
            
#         if len(raw_data) >= 2:
#             from tamalero.FIFO import merge_words
#             merged = merge_words(raw_data[:20]) 
                
#             print(f"Merged {len(merged)} events:")
#             for j, word in enumerate(merged[:10]):
#                 data_type, event_data = df.read(word)
                
#                 print(f"  Event {j}: {data_type} -> {event_data}")
            
#     except Exception as e:
#         print(f"Read failed: {e}")

    


=== Get data without charge injection ===
4
4
4
4
4
SUCCESS: Data is being generated!
   FIFO returned 9258 data items
  Event 0: header -> {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 977, 'raw': '0x3c5c1383d1', 'raw_full': '0x30c3c5c1383d1', 'meta': '0x30c'}
  Event 1: header -> {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 977, 'raw': '0x3c5c1383d1', 'raw_full': '0x30c3c5c1383d1', 'meta': '0x30c'}
  Event 2: header -> {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 977, 'raw': '0x3c5c1383d1', 'raw_full': '0x30c3c5c1383d1', 'meta': '0x30c'}
  Event 3: header -> {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 669, 'raw': '0x3c5c13829d', 'raw_full': '0x30c3c5c13829d', 'meta': '0x30c'}
  Event 4: header -> {'elink': 12, '

In [9]:
onchip_l1a_value = my_etroc2.rd_reg("onChipL1AConf")
print(f"onChipL1AConf  value: {onchip_l1a_value}")   ## should be 0


4
onChipL1AConf  value: 0


Reproduce ETROC data with charge injection, should see data type == data(hit)

In [ ]:
import numpy as np
import random
from datetime import datetime, timezone
import os
import pickle


print("\n=== Charge Injection test ===")

my_etroc2.reset()
my_etroc2.wr_reg("disDataReadout", 1, broadcast=True)

pixel_row = random.randint(0, 16)  
pixel_col = random.randint(0, 16)  
L1Adelay = 501           
charge = 30     

print(f"Pixel: Row {pixel_row}, Col {pixel_col}")
print(f"L1A Delay: {L1Adelay}")
print(f"Charges: {charge} fC")


my_etroc2.wr_reg("disDataReadout", 0, row=pixel_row, col=pixel_col, broadcast=False)
# The QSel register takes a value from 0-31 to represent 1-32 fC
my_etroc2.wr_reg("QSel", charge - 1, row=pixel_row, col=pixel_col)
my_etroc2.wr_reg("QInjEn", 1, row=pixel_row, col=pixel_col) #enable charge injection

fifo.reset()
fifo_depth = 8192*2  # number of words the FIFO can hold
events_max = int(fifo_depth/4)  # calculate the maximum number if l1as that the FIFO can handle
print(f"Sending {events_max} L1As and charge injection pulses")
fifo.send_QInj(events_max, delay=my_etroc2.QINJ_delay)

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    print(f"   FIFO returned {occupancy} data items")

    output_dir = "Qinj_Output"
    os.makedirs(output_dir, exist_ok=True)

    if occupancy > 0:
        print(green("   SUCCESS: Data was generated!"))

        test_results = {
            "test_parameters": {
                "timestamp_utc": datetime.now(timezone.utc).isoformat(),
                "pixel_row": pixel_row,
                "pixel_col": pixel_col,
                "L1A_delay": L1Adelay,
                "charge_fC": charge,
                "l1a_sent": events_max
            },
            "run_summary": {},
            "analysis": {},
            "parsed_hits": [], # Storing all hit data dictionaries
            "raw_events": data
        }
        
        # Parse the data
        header_count = 0
        hit_counter = 0
        toa_values = []
        tot_values = []
        cal_values = []

        print("\n--- Parsed Hit Data ---")
        for i, event in enumerate(data[:10]):  # printing first 10 to console
            print(event)
            data_type, event_data = event[0], event[1]
            
            if data_type == 'header':
                header_count += 1
            elif data_type == 'data':
                hit_counter += 1
                test_results["parsed_hits"].append(event_data) # Add full hit data 
      
                # Extract values
                toa = int(event_data.get('toa'))
                tot = int(event_data.get('tot'))
                cal = int(event_data.get('cal'))
                row = int(event_data.get('row_id'))
                col = int(event_data.get('col_id'))
                
                if toa is not None: toa_values.append(toa)
                if tot is not None: tot_values.append(tot)
                if cal is not None: cal_values.append(cal)

                # Print the first 20 hits to the console
                if hit_counter <= 20: 
                    if row == pixel_row and col == pixel_col:
                        print(green(f"  Hit {hit_counter:>2}: Pixel(R{row}, C{col}) -> ToA: {toa:<4}, ToT: {tot:<4}, Cal: {cal:<4}"))
                    else:
                        print(yellow(f"  Hit {hit_counter:>2}: UNEXPECTED Pixel(R{row}, C{col}) -> ToA: {toa:<4}, ToT: {tot:<4}, Cal: {cal:<4}"))

        print(f"\n--- Summary ---")
        print(f"Total events processed: {len(data)}")
        print(f"Headers found: {header_count}")
        print(f"Hits detected: {hit_counter}")

        # Populate and save the full results to file
        test_results["run_summary"]["total_events_processed"] = len(data)
        test_results["run_summary"]["headers_found"] = header_count
        test_results["run_summary"]["hits_detected"] = hit_counter

        if toa_values:
            avg_toa = np.mean(toa_values)
            test_results["analysis"]["avg_toa"] = avg_toa
            print(f"ToA values: {len(toa_values)} numeric values, avg = {avg_toa:.2f}")
        if tot_values:
            avg_tot = np.mean(tot_values)
            test_results["analysis"]["avg_tot"] = avg_tot
            print(f"ToT values: {len(tot_values)} numeric values, avg = {avg_tot:.2f}")
        if cal_values:
            avg_cal = np.mean(cal_values)
            test_results["analysis"]["avg_cal"] = avg_cal
            print(f"Cal values: {len(cal_values)} numeric values, avg = {avg_cal:.2f}")
            
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        ## Not using json due to standard json can't serialize Numpy data types
        pickle_filename = f"charge_injection_R{pixel_row}_C{pixel_col}_{timestamp}.pkl"
        pickle_filepath = os.path.join(output_dir, pickle_filename)
        
        try:
            with open(pickle_filepath, 'wb') as f:
                pickle.dump(test_results, f)
            print(green("Successfully saved pickle file."))
            print(green("Successfully saved results."))
        except Exception as e:
            print(f"Error saving file: {e}")
        
            
    else:
        print(red("FIFO returned no data. No hits detected."))
        
except Exception as e:
    print("Exception ", e)


print("\nCleaning up...")
my_etroc2.disable_QInj(row=pixel_row, col=pixel_col, broadcast=False)
my_etroc2.wr_reg("QInjEn", 0, row=pixel_row, col=pixel_col)

my_etroc2.wr_reg("disDataReadout", 0, broadcast=True)  # Re-enable all pixels
print("Test complete!")


=== Charge Injection test ===
4
Pixel: Row 14, Col 3
L1A Delay: 501
Charges: 30 fC
4
4
4
Sending 4096 L1As and charge injection pulses
   FIFO returned 16384 data items
   SUCCESS: Data was generated!

--- Parsed Hit Data ---
('header', {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 2315, 'raw': '0x3c5c13890b', 'raw_full': '0x30c3c5c13890b', 'meta': '0x30c'})
('header', {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 2315, 'raw': '0x3c5c13890b', 'raw_full': '0x30c3c5c13890b', 'meta': '0x30c'})
('header', {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 2315, 'raw': '0x3c5c13890b', 'raw_full': '0x30c3c5c13890b', 'meta': '0x30c'})
('data', {'ea': 0, 'col_id': 3, 'row_id': 14, 'toa': 243, 'cal': 169, 'tot': 37, 'elink': 12, 'full': 0, 'any_full': 0, 'global_full': 0, 'raw': '0x87c79894a9', 'ra

In [29]:
import pickle

with open("Qinj_Output/charge_injection_R14_C3_2025-06-13_02-08-32.pkl", 'rb') as f:
    data = pickle.load(f)

print("=== ALL PARAMETERS ===\n")

for main_key in ['test_parameters', 'run_summary', 'analysis', 'parsed_hits', 'raw_events']:
    print(f" {main_key.upper().replace('_', ' ')}:")
    
    if main_key in data:
        if main_key == 'parsed_hits':
            print(f"   Count: {len(data[main_key])}")
            for i, hit in enumerate(data[main_key]):
                print(f"   Hit {i+1}: {hit}")
                
        elif main_key == 'raw_events':
            print(f"   Count: {len(data[main_key])}")
            # Show first 10 as examples
            for i, event in enumerate(data[main_key][:10]):
                print(f"   Event {i+1}: {event}")
            if len(data[main_key]) > 10:
                print(f"   ... and {len(data[main_key])-10} more events")
        
        # check if data is a dictionary
        elif isinstance(data[main_key], dict):
            for key, value in data[main_key].items():
                print(f"   {key}: {value}")
        else:
            print(f"   {data[main_key]}")
    else:
        print("   Not found")
    
    print()  # Empty line between sections

=== ALL PARAMETERS ===

 TEST PARAMETERS:
   timestamp_utc: 2025-06-13T07:08:32.301436+00:00
   pixel_row: 14
   pixel_col: 3
   L1A_delay: 501
   charge_fC: 30
   l1a_sent: 4096

 RUN SUMMARY:
   total_events_processed: 16384
   headers_found: 3
   hits_detected: 2

 ANALYSIS:
   avg_toa: 243.0
   avg_tot: 37.0
   avg_cal: 169.0

 PARSED HITS:
   Count: 2
   Hit 1: {'ea': 0, 'col_id': 3, 'row_id': 14, 'toa': 243, 'cal': 169, 'tot': 37, 'elink': 12, 'full': 0, 'any_full': 0, 'global_full': 0, 'raw': '0x87c79894a9', 'raw_full': '0x30c87c79894a9', 'meta': '0x30c'}
   Hit 2: {'ea': 0, 'col_id': 3, 'row_id': 14, 'toa': 243, 'cal': 169, 'tot': 37, 'elink': 12, 'full': 0, 'any_full': 0, 'global_full': 0, 'raw': '0x87c79894a9', 'raw_full': '0x30c87c79894a9', 'meta': '0x30c'}

 RAW EVENTS:
   Count: 16384
   Event 1: ('header', {'elink': 12, 'sof': 1, 'eof': 1, 'full': 0, 'any_full': 0, 'global_full': 0, 'l1counter': 78, 'type': 0, 'bcid': 2315, 'raw': '0x3c5c13890b', 'raw_full': '0x30c3c5c138